In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_ = pd.read_csv("/Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/out/MASLAKTEPE_RES_birlesik_2024.csv")
df = df_.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Tarih                 8784 non-null   object 
 1   Ay                    8784 non-null   int64  
 2   Saat                  8784 non-null   object 
 3   PTF                   8784 non-null   float64
 4   SMF                   8784 non-null   float64
 5   Poz_DF                8784 non-null   float64
 6   Neg_DF                8784 non-null   float64
 7   KGUP                  8784 non-null   float64
 8   URETIM                8784 non-null   float64
 9   Dengesizlik           8784 non-null   float64
 10  GOP_Geliri            8784 non-null   float64
 11  Dengesizlik_Tutarı    8784 non-null   float64
 12  Net_Gelir             8784 non-null   float64
 13  Dengesizlik_Maliyeti  8784 non-null   float64
 14  Birim_DM              8784 non-null   float64
dtypes: float64(12), int64

In [4]:
df.head()

,Tarih,Ay,Saat,PTF,SMF,Poz_DF,Neg_DF,KGUP,URETIM,Dengesizlik,GOP_Geliri,Dengesizlik_Tutarı,Net_Gelir,Dengesizlik_Maliyeti,Birim_DM
0,2024-01-01,1,00:00,1299.98,560.00,543.2000,1338.9794,0.8,0.00,-0.80,1039.984,-1071.183520,-31.199520,31.199520,0.000000
1,2024-01-01,1,01:00,1299.98,1299.98,1260.9806,1338.9794,1.1,6.71,5.61,1429.978,7074.101166,8504.079166,218.786634,32.606056
2,2024-01-01,1,02:00,1248.54,1248.54,1211.0838,1285.9962,1.7,9.34,7.64,2122.518,9252.680232,11375.198232,286.165368,30.638690
3,2024-01-01,1,03:00,1299.98,1250.00,1212.5000,1338.9794,2.4,12.87,10.47,3119.952,12694.875000,15814.827000,915.915600,71.166713
4,2024-01-01,1,04:00,1200.00,1150.00,1115.5000,1236.0000,2.8,9.22,6.42,3360.000,7161.510000,10521.510000,542.490000,58.838395


In [5]:
df['Tarih'] = pd.to_datetime(df['Tarih'])
print(f"\nDönüşüm tamamlandı!")
print(f"Yeni veri tipi: {df['Tarih'].dtype}")
print("Örnek değerler:")
df['Tarih'].head()


Dönüşüm tamamlandı!
Yeni veri tipi: datetime64[ns]
Örnek değerler:


0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-01-01
4   2024-01-01
Name: Tarih, dtype: datetime64[ns]

In [9]:
print("=== Veri Kalite Kontrolü ===")
print(f"Toplam satır: {len(df)}")
print(f"Eksik değerler: {df.isnull().sum().sum()}")
print(f"Duplikat satırlar: {df.duplicated().sum()}")

# Saat kontrolü (24 saat × 366 gün = 8784)
print(f"\n=== Saat Dağılımı ===")
print(f"Benzersiz saat sayısı: {df['Saat'].nunique()}")
print(f"Benzersiz tarih sayısı: {df['Tarih'].nunique()}")

# Değer aralıkları
print(f"\n=== Değer Aralıkları ===")
print(f"PTF: {df['PTF'].min():.2f} - {df['PTF'].max():.2f}")
print(f"SMF: {df['SMF'].min():.2f} - {df['SMF'].max():.2f}")
print(f"KGÜP: {df['KGUP'].min():.2f} - {df['KGUP'].max():.2f}")
print(f"Üretim: {df['URETIM'].min():.2f} - {df['URETIM'].max():.2f}")

# Dengesizlik maliyeti özeti
print(f"\n=== Dengesizlik Maliyeti ===")
print(f"Toplam DM: {df['Dengesizlik_Maliyeti'].sum():.2f} TL")
print(f"Ortalama Birim DM: {df['Birim_DM'].mean():.4f} TL/MWh")
print(f"Toplam Üretim: {df['URETIM'].sum():.2f} MWh")
print("*" * 20)
print("NET GELİR: ", df["Net_Gelir"].sum())

=== Veri Kalite Kontrolü ===
Toplam satır: 8784
Eksik değerler: 0
Duplikat satırlar: 0

=== Saat Dağılımı ===
Benzersiz saat sayısı: 24
Benzersiz tarih sayısı: 366

=== Değer Aralıkları ===
PTF: 0.00 - 3000.00
SMF: 0.00 - 3000.00
KGÜP: 0.00 - 68.40
Üretim: 0.00 - 68.39

=== Dengesizlik Maliyeti ===
Toplam DM: 17054420.03 TL
Ortalama Birim DM: 142.0582 TL/MWh
Toplam Üretim: 228563.03 MWh
********************
NET GELİR:  465277004.259199


In [8]:
df[df["URETIM"] < 0].shape

(0, 15)

In [10]:
# Aylık agregasyon ve temel performans metrikleri
df_monthly = df.groupby('Ay').agg({
    'KGUP': 'sum',
    'URETIM': 'sum', 
    'Dengesizlik': 'sum',
    'GOP_Geliri': 'sum',
    'Dengesizlik_Tutarı': 'sum',
    'Net_Gelir': 'sum',
    'Dengesizlik_Maliyeti': 'sum'
}).round(2)

# Aylık performans metrikleri
df_monthly['Dengesizlik_Orani'] = (df_monthly['Dengesizlik'] / df_monthly['KGUP'] * 100).round(4)
df_monthly['Birim_DM_Aylik'] = (df_monthly['Dengesizlik_Maliyeti'] / df_monthly['URETIM']).round(4)

# Kapasite faktörü hesaplama (düzeltilmiş)
days_per_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]  # 2024 artık yıl
max_possible_production = [62.4 * 24 * days for days in days_per_month]
df_monthly['Kapasite_Faktoru'] = (df_monthly['URETIM'] / max_possible_production * 100).round(2)

df_monthly['DM_Gelir_Orani'] = (df_monthly['Dengesizlik_Maliyeti'] / df_monthly['Net_Gelir'] * 100).round(4)

print("=== MASLAKTEPE RES - Aylık Performans Özeti ===")
df_monthly

=== MASLAKTEPE RES - Aylık Performans Özeti ===


,KGUP,URETIM,Dengesizlik,GOP_Geliri,Dengesizlik_Tutarı,Net_Gelir,Dengesizlik_Maliyeti,Dengesizlik_Orani,Birim_DM_Aylik,Kapasite_Faktoru,DM_Gelir_Orani
Ay,,,,,,,,,,,
1,26726.70,26564.30,-162.40,48272332.20,-1284524.23,46987807.97,1382658.92,-0.6076,52.0495,57.22,2.9426
2,16454.30,16909.42,455.12,28653745.37,-842364.91,27811380.47,1534771.86,2.7660,90.7643,38.93,5.5185
3,9078.40,9718.56,640.16,18671961.63,160037.03,18831998.66,1239251.13,7.0515,127.5139,20.93,6.5806
4,20309.60,19975.01,-334.59,30579630.56,-2799752.85,27779877.72,1811767.07,-1.6474,90.7017,44.46,6.5219
5,13313.62,14419.76,1106.14,23629667.93,1028877.74,24658545.67,1178785.28,8.3083,81.7479,31.06,4.7804
6,22458.58,21884.61,-573.97,44956781.17,-2116823.34,42839957.83,1166129.58,-2.5557,53.2854,48.71,2.7221
7,27725.80,26088.38,-1637.42,70276102.09,-4994222.07,65281880.02,962859.51,-5.9058,36.9076,56.19,1.4749
8,22030.50,22050.65,20.15,55651380.92,-858795.26,54792585.66,1079472.86,0.0915,48.9542,47.50,1.9701
9,10016.50,10653.75,637.25,23155905.98,250405.35,23406311.32,1397129.20,6.3620,131.1397,23.71,5.9690


Maslaktepe RES'in 2024 yılı performansı Eber RES'e kıyasla çok daha başarılı bir profil sergilemiştir. Yaz ayları (Haziran-Ağustos) operasyonel mükemmelliğin doruk noktası olmuştur; özellikle Temmuz ayında %56.19 kapasite faktörü ile en yüksek üretim gerçekleştirilirken, 36.91 TL/MWh ile en düşük birim dengesizlik maliyeti ve sadece %1.47 maliyet-gelir oranı ile sektörde örnek teşkil edecek bir performans gösterilmiştir. Ağustos ayında ise neredeyse mükemmel bir dengesizlik oranı (%0.09) yakalanarak tahmin sistemlerinin ne kadar etkili çalıştığı kanıtlanmıştır.
Kış döneminde bile Maslaktepe RES oldukça kontrollü bir performans sergilemiştir. Ocak ayında %57.22 ile en yüksek kapasite faktörüne ulaşılırken, dengesizlik oranı sadece %-0.61 olarak kalarak Eber RES'teki dramatik dalgalanmaların aksine istikrarlı bir üretim sağlanmıştır. En zorlu dönem Mart ayı olmuştur: %20.93 ile en düşük kapasite faktörü ve 127.51 TL/MWh ile en yüksek birim dengesizlik maliyeti yaşanmıştır.
Finansal açıdan Maslaktepe RES, 465.3 milyon TL net gelir ile Eber RES'ten sadece %9.8 daha fazla gelir elde etmiştir. Ancak asıl fark dengesizlik yönetiminde ortaya çıkmaktadır: Maslaktepe RES'in 17.05 milyon TL dengesizlik maliyeti, Eber RES'in 20.9 milyon TL'lik maliyetinden 3.85 milyon TL daha düşüktür. Bu durum, benzer gelir seviyelerine rağmen Maslaktepe RES'in dengesizlik kontrolünde daha başarılı olduğunu göstermektedir. Maliyet-gelir oranı açısından da Maslaktepe RES daha istikrarlı performans sergilemiş, hiçbir ayda %7'yi aşan oran görülmezken çoğu ayda %5'in altında kalınmıştır. Bu performans, aynı gelir seviyesindeki rüzgar santrallerinde operasyonel mükemmelliğin dengesizlik maliyetlerini önemli ölçüde azaltabileceğini ve karlılığı artırabileceğini kanıtlamaktadır.

In [11]:
df_monthly["DM_Gelir_Orani"].mean()

np.float64(4.202875)

In [14]:
print("=== MASLAKTEPE RES - Risk Analizi ===")

# En yüksek DM yaşanan günler
daily_dm = df.groupby('Tarih')['Dengesizlik_Maliyeti'].sum()
print("En yüksek dengesizlik maliyeti yaşanan 10 gün:")
# En yüksek dengesizlik maliyeti yaşanan 10 günü, önce DM'ye göre en yüksekten seçip, sonra tarihe göre (en yeni gün en üstte) göstermek için:
top10_dm_days = daily_dm.sort_values(ascending=False).head(10)
top10_dm_days = top10_dm_days.sort_index(ascending=False)
print(top10_dm_days)

# En yüksek maliyetli günlerin toplam istatistikleri
print(f"\nEn Maliyetli 10 Gün - Genel İstatistikler:")
print(f"Toplam Maliyet: {top10_dm_days.sum():,.0f} TL")
print(f"Ortalama Maliyet: {top10_dm_days.mean():,.0f} TL/gün")
print(f"Toplam Maliyetteki Payı: {top10_dm_days.sum() / df['Dengesizlik_Maliyeti'].sum() * 100:.1f}%")

hourly_analysis = df.groupby('Saat').agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Dengesizlik': 'mean',
    'PTF': 'mean',
    'KGUP': 'mean'
}).reset_index()

# Dengesizlik oranı = dengesizlik / planlanan üretim
hourly_analysis['Dengesizlik_Orani'] = (
    hourly_analysis['Dengesizlik'] / hourly_analysis['KGUP'] * 100
).round(2)

# En riskli saatler
risky_hours = hourly_analysis.nlargest(5, 'Dengesizlik_Orani')[['Dengesizlik_Maliyeti', 'Dengesizlik_Orani', 'PTF']]
print(f"\nEn riskli 5 saat:")
print(risky_hours)

# Mevsimsel risk analizi
seasonal_risk = df.groupby('Ay').agg({
    'Dengesizlik_Maliyeti': ['sum', 'mean', 'std'],
    'Birim_DM': 'mean'
}).round(2)
seasonal_risk.columns = ['Toplam_DM', 'Ortalama_DM', 'DM_Volatilite', 'Birim_DM']
seasonal_risk = seasonal_risk.sort_values('Birim_DM', ascending=False)
print(f"\nMevsimsel Risk Profili (Birim_DM'ye göre büyükten küçüğe sıralı):")
print(seasonal_risk)

# Extreme event analizi
extreme_dm = df[df['Dengesizlik_Maliyeti'] > df['Dengesizlik_Maliyeti'].quantile(0.95)]
print(f"\nExtreme DM Events (>95. persentil):")
print(f"Toplam olay sayısı: {len(extreme_dm)}")
print(f"Bu olayların ortalama PTF'si: {extreme_dm['PTF'].mean():.0f} TL/MWh")
print(f"Bu olayların ortalama dengesizlik büyüklüğü: {extreme_dm['Dengesizlik'].abs().mean():.2f} MWh")

=== MASLAKTEPE RES - Risk Analizi ===
En yüksek dengesizlik maliyeti yaşanan 10 gün:
Tarih
2024-12-26    190456.531664
2024-12-10    415163.028126
2024-12-07    223082.379910
2024-10-31    153070.801899
2024-09-01    153232.830520
2024-05-15    216894.972471
2024-03-28    205998.815612
2024-02-09    162380.975511
2024-02-07    256732.142338
2024-01-06    277952.716972
Name: Dengesizlik_Maliyeti, dtype: float64

En Maliyetli 10 Gün - Genel İstatistikler:
Toplam Maliyet: 2,254,965 TL
Ortalama Maliyet: 225,497 TL/gün
Toplam Maliyetteki Payı: 13.2%

En riskli 5 saat:
    Dengesizlik_Maliyeti  Dengesizlik_Orani          PTF
18           1398.543707               5.25  2702.840546
19           1325.688426               3.76  2727.399399
20           1825.009420               3.58  2726.846175
22           2502.292620               3.58  2416.326557
21           1815.630557               3.32  2652.683907

Mevsimsel Risk Profili (Birim_DM'ye göre büyükten küçüğe sıralı):
     Toplam_DM  Ortal

In [13]:
# En maliyetli 30 saat detay analizi
print("=== MASLAKTEPE RES - En Maliyetli 30 Saat Detay Analizi ===")

# En yüksek dengesizlik maliyeti olan 30 saati seç
top_30_cost = df.nlargest(30, 'Dengesizlik_Maliyeti').copy()
top_30_cost['PTF_SMF_Spread'] = abs(top_30_cost['PTF'] - top_30_cost['SMF'])

print(f"En Maliyetli 30 Saat - Genel İstatistikler:")
print(f"Toplam Maliyet: {top_30_cost['Dengesizlik_Maliyeti'].sum():,.0f} TL")
print(f"Ortalama Maliyet: {top_30_cost['Dengesizlik_Maliyeti'].mean():,.0f} TL/saat")
print(f"Toplam Maliyetteki Payı: {top_30_cost['Dengesizlik_Maliyeti'].sum() / df['Dengesizlik_Maliyeti'].sum() * 100:.1f}%")

# Zaman dağılımı
time_distribution = top_30_cost.groupby('Saat').size().sort_values(ascending=False)
print(f"\nEn Maliyetli Saatlerin Zaman Dağılımı:")
print(time_distribution.head())

month_distribution = top_30_cost.groupby('Ay').size().sort_values(ascending=False)
print(f"\nEn Maliyetli Saatlerin Ay Dağılımı:")
print(month_distribution.head())

# Ortak karakteristikler
print(f"\nEn Maliyetli 30 Saatin Karakteristikleri:")
characteristics = top_30_cost.agg({
    'PTF': ['mean', 'min', 'max'],
    'SMF': ['mean', 'min', 'max'],
    'URETIM': ['mean', 'min', 'max'],
    'KGUP': ['mean', 'min', 'max'],
    'Dengesizlik': ['mean', 'min', 'max'],
    'PTF_SMF_Spread': ['mean', 'min', 'max'] if 'PTF_SMF_Spread' in df.columns else ['mean']
}).round(2)

print(characteristics)

# Bu saatlerdeki dengesizlik yönü
positive_imbalance = (top_30_cost['Dengesizlik'] > 0).sum()
negative_imbalance = (top_30_cost['Dengesizlik'] < 0).sum()
print(f"\nDengesizlik Yönü Dağılımı:")
print(f"Pozitif (Fazla Üretim): {positive_imbalance} saat")
print(f"Negatif (Az Üretim): {negative_imbalance} saat")

# Detaylı liste (ilk 10 saat)
print(f"\nEn Maliyetli 10 Saat Detayı:")
detailed_top10 = top_30_cost.head(10)[['Tarih', 'Saat', 'PTF', "SMF", 'KGUP', 'URETIM', 'Dengesizlik', 'Dengesizlik_Maliyeti']]
detailed_top10

=== MASLAKTEPE RES - En Maliyetli 30 Saat Detay Analizi ===
En Maliyetli 30 Saat - Genel İstatistikler:
Toplam Maliyet: 1,339,520 TL
Ortalama Maliyet: 44,651 TL/saat
Toplam Maliyetteki Payı: 7.9%

En Maliyetli Saatlerin Zaman Dağılımı:
Saat
20:00    4
22:00    3
21:00    3
01:00    2
19:00    2
dtype: int64

En Maliyetli Saatlerin Ay Dağılımı:
Ay
12    8
1     4
2     4
10    3
3     2
dtype: int64

En Maliyetli 30 Saatin Karakteristikleri:
          PTF      SMF  URETIM  KGUP  Dengesizlik  PTF_SMF_Spread
mean  2473.75   960.44   54.00  28.2        25.80         1583.92
min   1631.98   400.00    1.75   4.2       -41.45             NaN
max   3000.00  2691.09   68.37  48.0        44.83             NaN

Dengesizlik Yönü Dağılımı:
Pozitif (Fazla Üretim): 29 saat
Negatif (Az Üretim): 1 saat

En Maliyetli 10 Saat Detayı:


,Tarih,Saat,PTF,SMF,KGUP,URETIM,Dengesizlik,Dengesizlik_Maliyeti
8276,2024-12-10,20:00,2830.55,1000.00,13.6,53.59,39.99,74403.394500
8275,2024-12-10,19:00,2724.96,1351.00,14.7,59.53,44.83,63411.586700
958,2024-02-09,22:00,2299.15,630.00,30.2,66.08,35.88,60567.234000
8204,2024-12-07,20:00,2780.01,625.00,36.5,64.23,27.73,60278.364800
649,2024-01-28,01:00,2324.99,800.00,26.2,61.19,34.99,54199.160100
130,2024-01-06,10:00,2380.00,630.00,20.0,50.43,30.43,53827.627000
7254,2024-10-29,06:00,2500.01,700.00,26.2,53.71,27.51,50095.985100
8649,2024-12-26,09:00,3000.00,1000.00,42.8,66.97,24.17,49065.100000
2772,2024-04-25,12:00,1631.98,2691.09,43.2,1.75,-41.45,47246.479915
8205,2024-12-07,21:00,2675.00,600.00,38.4,60.48,22.08,46213.440000


In [36]:
STEP = 7.0

def optimize_kgnp_combined(df, risk_aylar, risk_saatler, maliyet_aylar, maliyet_saatler):
    """
    Kesişen zamanlar için risk stratejisi, kalanlar için maliyet stratejisi uygular
    """
    df_opt = df.copy()
    
    # Saat string → int
    if df_opt['Saat'].dtype == 'object':
        df_opt['Saat'] = pd.to_datetime(df_opt['Saat'], format='%H:%M').dt.hour
    
    df_opt['KGUP_Opt'] = df_opt['KGUP']
    df_opt['Strateji'] = 'Değişiklik_Yok'
    
    # Risk ve maliyet kümelerini tanımla
    risk_set = set((ay, saat) for ay in risk_aylar for saat in risk_saatler)
    maliyet_set = set((ay, saat) for ay in maliyet_aylar for saat in maliyet_saatler)
    
    # Kesişenleri bul
    kesisen_set = risk_set.intersection(maliyet_set)
    
    # Sadece risk setinde olanlar (kesişenler hariç)
    sadece_risk_set = risk_set - kesisen_set
    
    # Sadece maliyet setinde olanlar (kesişenler hariç)
    sadece_maliyet_set = maliyet_set - kesisen_set
    
    print(f"Kesişen zaman dilimleri: {len(kesisen_set)}")
    print(f"Sadece risk stratejisi: {len(sadece_risk_set)}")
    print(f"Sadece maliyet stratejisi: {len(sadece_maliyet_set)}")
    print(f"Toplam optimize edilecek kombinasyon: {len(kesisen_set) + len(sadece_risk_set) + len(sadece_maliyet_set)}")
    
    # Optimizasyon uygula
    for idx, row in df_opt.iterrows():
        ay_saat = (row['Ay'], row['Saat'])
        
        # Strateji uygulanacak mı kontrol et
        if ay_saat in kesisen_set or ay_saat in sadece_risk_set or ay_saat in sadece_maliyet_set:
            
            # Strateji adını belirle
            if ay_saat in kesisen_set or ay_saat in sadece_risk_set:
                 df_opt.at[idx, 'Strateji'] = 'Risk_Stratejisi'
            else: # sadece_maliyet_set
                 df_opt.at[idx, 'Strateji'] = 'Maliyet_Stratejisi'

            # Ortak optimizasyon mantığını uygula
            kgup = row['KGUP_Opt']
            uretim = row['URETIM']
            ptf = row['PTF']
            poz = row['Poz_DF']
            neg = row['Neg_DF']
            
            d = uretim - kgup
            
            if d < 0:  # eksik üretim
                if neg > ptf:
                    kgup -= STEP
            else:      # fazla üretim
                if poz < ptf:
                    kgup += STEP
            
            if kgup < 0:
                kgup = 0
            df_opt.at[idx, 'KGUP_Opt'] = kgup
    
    # Gelir hesaplamaları
    df_opt['GOP_Opt'] = df_opt['KGUP_Opt'] * df_opt['PTF']
    df_opt['Dengesizlik_Opt'] = df_opt['URETIM'] - df_opt['KGUP_Opt']
    df_opt['Fiyat_Opt'] = df_opt.apply(
        lambda row: row['Poz_DF'] if row['Dengesizlik_Opt'] >= 0 else row['Neg_DF'], axis=1
    )
    df_opt['Dengesizlik_Tutarı_Opt'] = df_opt['Dengesizlik_Opt'] * df_opt['Fiyat_Opt']
    df_opt['Net_Opt'] = df_opt['GOP_Opt'] + df_opt['Dengesizlik_Tutarı_Opt']
    
    # Sonuçlar
    toplam_mevcut = df['Net_Gelir'].sum()
    toplam_yeni = df_opt['Net_Opt'].sum()
    iyilestirme = toplam_yeni - toplam_mevcut
    
    print(f"\n=== SONUÇLAR ===")
    print(f"Mevcut Net Gelir: {toplam_mevcut:,.0f} TL")
    print(f"Optimize Net Gelir: {toplam_yeni:,.0f} TL")
    print(f"Toplam İyileştirme: {iyilestirme:,.0f} TL")
    
    # Strateji bazında detay
    strateji_detay = df_opt.groupby('Strateji').agg({
        'Net_Gelir': 'sum',
        'Net_Opt': 'sum'
    })
    strateji_detay['Iyilestirme'] = strateji_detay['Net_Opt'] - strateji_detay['Net_Gelir']
    
    print(f"\n=== STRATEJİ BAZINDA DETAY ===")
    print(strateji_detay)
    
    return df_opt

df_opt_combined = optimize_kgnp_combined(
    df, 
    risk_aylar=[1,2,3,5,9,10,12], 
    risk_saatler=[18,19,20,21,22],
    maliyet_aylar=[1,2,3,10,12], 
    maliyet_saatler=[1,19,20,21,22]
)

Kesişen zaman dilimleri: 20
Sadece risk stratejisi: 15
Sadece maliyet stratejisi: 5
Toplam optimize edilecek kombinasyon: 40

=== SONUÇLAR ===
Mevcut Net Gelir: 465,277,004 TL
Optimize Net Gelir: 466,222,565 TL
Toplam İyileştirme: 945,561 TL

=== STRATEJİ BAZINDA DETAY ===
                       Net_Gelir       Net_Opt    Iyilestirme
Strateji                                                     
Değişiklik_Yok      3.981152e+08  3.981152e+08       0.000000
Maliyet_Stratejisi  6.285686e+06  6.436354e+06  150667.830865
Risk_Stratejisi     6.087612e+07  6.167101e+07  794893.366362


In [29]:
STEP = 7.0

def optimize_kgnp(df, riskli_aylar, riskli_saatler, use_mean=True):
    """
    Belirlenen ay ve saatlere göre KGUP optimizasyonu yapar
    ve özet rapor döner.
    """
    df_opt = df.copy()

    # Saat string → int
    if df_opt['Saat'].dtype == 'object':
        df_opt['Saat'] = pd.to_datetime(df_opt['Saat'], format='%H:%M').dt.hour

    df_opt['KGUP_Opt'] = df_opt['KGUP']

    # --- optimizasyon ---
    for idx, row in df_opt.iterrows():
        if (row['Saat'] in riskli_saatler) and (row['Ay'] in riskli_aylar):
            kgup = row['KGUP_Opt']
            uretim = row['URETIM']
            ptf = row['PTF']
            poz = row['Poz_DF']
            neg = row['Neg_DF']

            d = uretim - kgup  # dengesizlik

            if d < 0:  # eksik üretim
                if neg > ptf:
                    kgup -= STEP
            else:      # fazla üretim
                if poz < ptf:
                    kgup += STEP

            if kgup < 0:
                kgup = 0

            df_opt.at[idx, 'KGUP_Opt'] = kgup

    # --- gelir hesaplamaları ---
    df_opt['GOP_Opt'] = df_opt['KGUP_Opt'] * df_opt['PTF']
    df_opt['Dengesizlik_Opt'] = df_opt['URETIM'] - df_opt['KGUP_Opt']
    df_opt['Fiyat_Opt'] = df_opt.apply(
        lambda row: row['Poz_DF'] if row['Dengesizlik_Opt'] >= 0 else row['Neg_DF'], axis=1
    )
    df_opt['Dengesizlik_Tutarı_Opt'] = df_opt['Dengesizlik_Opt'] * df_opt['Fiyat_Opt']
    df_opt['Net_Opt'] = df_opt['GOP_Opt'] + df_opt['Dengesizlik_Tutarı_Opt']

    # --- toplam sonuç ---
    toplam_mevcut = df['Net_Gelir'].sum()
    toplam_yeni = df_opt['Net_Opt'].sum()
    iyilestirme = toplam_yeni - toplam_mevcut

    print("Mevcut Net Gelir:", toplam_mevcut)
    print("Optimize Net Gelir:", toplam_yeni)
    print("İyileştirme:", iyilestirme)

    # --- detay rapor ---
    ay_ad_tr = {
        1: 'ocak', 2: 'şubat', 3: 'mart', 4: 'nisan', 5: 'mayıs', 6: 'haziran',
        7: 'temmuz', 8: 'ağustos', 9: 'eylül', 10: 'ekim', 11: 'kasım', 12: 'aralık'
    }

    agg_fn = np.mean if use_mean else np.sum
    label = "ort" if use_mean else "top"

    for ay in riskli_aylar:
        for saat in riskli_saatler:
            mask = (df['Ay'] == ay) & (df['Saat'] == saat)
            if mask.sum() == 0:
                continue

            kgup_mevcut = agg_fn(df.loc[mask, 'KGUP'])
            uretim_mevcut = agg_fn(df.loc[mask, 'URETIM'])
            kgup_yeni = agg_fn(df_opt.loc[mask, 'KGUP_Opt'])
            net_mevcut = agg_fn(df.loc[mask, 'Net_Gelir'])
            net_yeni = agg_fn(df_opt.loc[mask, 'Net_Opt'])
            delta_kar = net_yeni - net_mevcut

            print(
                f"ay: {ay_ad_tr.get(ay, ay)} saat: {saat:02d}:00 "
                f"mevcut KGUP({label}): {kgup_mevcut:.2f} "
                f"mevcut uretim({label}): {uretim_mevcut:.2f} "
                f"yeni KGUP({label}): {kgup_yeni:.2f} "
                f"kar_yeni({label}): {net_yeni:,.0f} "
                f"kar_artis({label}): {delta_kar:,.0f}"
            )

    return df_opt

In [30]:
df_opt_risk = optimize_kgnp(df, riskli_aylar=[1,2,3,5,9,10,12], riskli_saatler=[18,19,20,21,22])

Mevcut Net Gelir: 465277004.259199
Optimize Net Gelir: 466071897.625561
İyileştirme: 794893.3663619757


In [31]:
df_opt_maliyet = optimize_kgnp(df, riskli_aylar=[1,2,3,10,12], riskli_saatler=[1,19,20,21,22]) # maliyetli saatlerde KGUP optimizasyonu

Mevcut Net Gelir: 465277004.259199
Optimize Net Gelir: 465977349.342435
İyileştirme: 700345.0832359791


In [37]:
def kpi_summary(df_monthly, plant_name="Santral"):
    num_cols = df_monthly.select_dtypes(include='number').columns

    summary = pd.DataFrame({
        "SUM": df_monthly[num_cols].sum(),
        "MEAN": df_monthly[num_cols].mean()
    }).round(2)

    summary.index.name = f"{plant_name} - KPI"
    return summary

maslaktepe_summary = kpi_summary(df_monthly, "MASLAKTEPE_RES")
maslaktepe_summary

,SUM,MEAN
MASLAKTEPE_RES - KPI,,
KGUP,2.260626e+05,18838.55
URETIM,2.285630e+05,19046.92
Dengesizlik,2.500450e+03,208.37
GOP_Geliri,4.761929e+08,39682745.51
Dengesizlik_Tutarı,-1.091594e+07,-909661.83
Net_Gelir,4.652770e+08,38773083.69
Dengesizlik_Maliyeti,1.705442e+07,1421201.67
Dengesizlik_Orani,2.612000e+01,2.18
Birim_DM_Aylik,9.763400e+02,81.36


Yorum: Gerçek üretim, planlanandan biraz fazla. Dengesizlik oranı %2.18 ile oldukça düşük, bu da tahminlerin başarılı olduğunu gösteriyor.
Her üretilen MWh başına ortalama 81 TL dengesizlik maliyeti ödenmiş. Bu değer, piyasa koşullarına göre düşük-orta arası bir seviyede.
Dengesizlik maliyeti, toplam gelirin %4.2’si (DM_Gelir_Orani)

In [38]:
# Dengesizlik performans analizi
print("=== MASLAKTEPE RES - Dengesizlik Performans Analizi ===")
print(f"Ortalama Dengesizlik Oranı: {(df['Dengesizlik'].sum() / df['KGUP'].sum() * 100):.2f}%")
print(f"Pozitif Dengesizlik Oranı: {(df['Dengesizlik'] > 0).sum() / len(df) * 100:.1f}%")
print(f"Negatif Dengesizlik Oranı: {(df['Dengesizlik'] < 0).sum() / len(df) * 100:.1f}%")
print(f"Sıfır Üretim Saatleri: {(df['URETIM'] == 0).sum()} saat ({(df['URETIM'] == 0).sum()/len(df)*100:.1f}%)")

# Dengesizlik büyüklük dağılımı
dengesizlik_abs = df['Dengesizlik'].abs()
print(f"\nDengesizlik Büyüklük İstatistikleri:")
print(f"Medyan: {dengesizlik_abs.median():.3f} MWh")
print(f"Ortalama: {dengesizlik_abs.mean():.3f} MWh") 
print(f"95. Persentil: {dengesizlik_abs.quantile(0.95):.3f} MWh")
print(f"Maksimum: {dengesizlik_abs.max():.3f} MWh")

=== MASLAKTEPE RES - Dengesizlik Performans Analizi ===
Ortalama Dengesizlik Oranı: 1.11%
Pozitif Dengesizlik Oranı: 44.7%
Negatif Dengesizlik Oranı: 54.6%
Sıfır Üretim Saatleri: 1415 saat (16.1%)

Dengesizlik Büyüklük İstatistikleri:
Medyan: 3.920 MWh
Ortalama: 5.908 MWh
95. Persentil: 18.466 MWh
Maksimum: 58.260 MWh


In [39]:
def performance_nonzero(df, name):
    # Sadece üretim yapılan saatleri al
    active = df[df["URETIM"] > 0].copy()
    result = {"Santral": name, "Aktif_Saat": len(active)}

    # Tahmine göre oransal sapma
    valid = active[active["KGUP"] > 0]
    result["Ortalama_Dengesizlik_%"] = (valid["Dengesizlik"].abs() / valid["KGUP"]).mean() * 100
    result["Medyan_Dengesizlik_%"]   = (valid["Dengesizlik"].abs() / valid["KGUP"]).median() * 100

    # Ortalama bias (pozitif = fazla üretim)
    result["Bias_MWh"]     = active["Dengesizlik"].mean()
    result["Pozitif_%"]    = (active["Dengesizlik"] > 0).mean() * 100
    result["Negatif_%"]    = (active["Dengesizlik"] < 0).mean() * 100

    # Dengesizlik büyüklüğü istatistikleri
    result["Medyan_MWh"]   = active["Dengesizlik"].median()
    result["Ortalama_MWh"] = active["Dengesizlik"].mean()
    result["P95_MWh"]      = active["Dengesizlik"].quantile(0.95)
    result["Max_MWh"]      = active["Dengesizlik"].max()

    return result

maslaktepe_perf  = performance_nonzero(df, "MASLAKTEPE_RES")

pd.DataFrame([maslaktepe_perf]).round(3)

,Santral,Aktif_Saat,Ortalama_Dengesizlik_%,Medyan_Dengesizlik_%,Bias_MWh,Pozitif_%,Negatif_%,Medyan_MWh,Ortalama_MWh,P95_MWh,Max_MWh
0,MASLAKTEPE_RES,7369,44.034,21.579,0.721,53.304,46.641,0.48,0.721,15.87,58.26


In [40]:
# Finansal etki analizi
print("=== MASLAKTEPE RES - Finansal Etki Analizi ===")
total_net_revenue = df['Net_Gelir'].sum()
total_dm_cost = df['Dengesizlik_Maliyeti'].sum()
total_production = df['URETIM'].sum()

print(f"Toplam Net Gelir: {total_net_revenue:,.0f} TL")
print(f"Toplam Dengesizlik Maliyeti: {total_dm_cost:,.0f} TL")
print(f"DM'nin Net Gelir'deki Payı: {total_dm_cost/total_net_revenue*100:.2f}%")
print(f"Yıllık Birim Dengesizlik Maliyeti: {total_dm_cost/total_production:.2f} TL/MWh")

# Aylık finansal performans
monthly_financial = df_monthly[['Net_Gelir', 'Dengesizlik_Maliyeti', 'DM_Gelir_Orani']].copy()
print(f"\nEn yüksek DM/Gelir oranına sahip aylar:")
print(monthly_financial.nlargest(3, 'DM_Gelir_Orani'))

# PTF-SMF spread analizi
df['PTF_SMF_Spread'] = abs(df['PTF'] - df['SMF'])
spread_analysis = df.groupby(pd.cut(df['PTF_SMF_Spread'], bins=5)).agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(2)
print(f"\nPTF-SMF Spread'e göre DM analizi:")
spread_analysis

=== MASLAKTEPE RES - Finansal Etki Analizi ===
Toplam Net Gelir: 465,277,004 TL
Toplam Dengesizlik Maliyeti: 17,054,420 TL
DM'nin Net Gelir'deki Payı: 3.67%
Yıllık Birim Dengesizlik Maliyeti: 74.62 TL/MWh

En yüksek DM/Gelir oranına sahip aylar:
      Net_Gelir  Dengesizlik_Maliyeti  DM_Gelir_Orani
Ay                                                   
3   18831998.66            1239251.13          6.5806
4   27779877.72            1811767.07          6.5219
9   23406311.32            1397129.20          5.9690

PTF-SMF Spread'e göre DM analizi:


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_1865/1676252011.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spread_analysis = df.groupby(pd.cut(df['PTF_SMF_Spread'], bins=5)).agg({


,Dengesizlik_Maliyeti,Birim_DM
PTF_SMF_Spread,,
"(-2.695, 539.0]",927.35,67.30
"(539.0, 1078.0]",3251.86,371.45
"(1078.0, 1617.0]",5816.77,202.78
"(1617.0, 2156.0]",10698.22,314.70
"(2156.0, 2695.0]",10638.26,248.75


In [41]:
# PTF-SMF spread analizi (sadece üretim yapılan saatler)
df_nonzero = df[df['URETIM'] > 0].copy()   # üretim yapılan saatler

df_nonzero['PTF_SMF_Spread_NonZero'] = abs(df_nonzero['PTF'] - df_nonzero['SMF'])
spread_analysis = df_nonzero.groupby(pd.cut(df_nonzero['PTF_SMF_Spread_NonZero'], bins=5)).agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(2)

print("\nPTF-SMF Spread'e göre DM analizi (sadece üretim yapılan saatler):")
spread_analysis


PTF-SMF Spread'e göre DM analizi (sadece üretim yapılan saatler):


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_1865/2166594354.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spread_analysis = df_nonzero.groupby(pd.cut(df_nonzero['PTF_SMF_Spread_NonZero'], bins=5)).agg({


,Dengesizlik_Maliyeti,Birim_DM
PTF_SMF_Spread_NonZero,,
"(-2.695, 539.0]",1042.92,81.35
"(539.0, 1078.0]",3621.57,428.98
"(1078.0, 1617.0]",6623.03,235.26
"(1617.0, 2156.0]",11865.71,352.12
"(2156.0, 2695.0]",11439.39,267.89


In [44]:
def melkom_dengesizlik_yonu_analizi(df):
    """
    MASLAKTEPE RES için pozitif vs negatif dengesizlik analizi
    Hangi yönde hata yapmak daha pahalı?
    """
    
    print("=" * 70)
    print("MASLAKTEPE RES - POZİTİF vs NEGATİF DENGESİZLİK ANALİZİ")
    print("=" * 70)
    
    # Dengesizlik yönlerini kategorize et
    df_analysis = df.copy()
    df_analysis['dengesizlik_yonu'] = 'Nötr'
    df_analysis.loc[df_analysis['Dengesizlik'] > 0, 'dengesizlik_yonu'] = 'Pozitif_Dengesizlik'  # Fazla ürettik
    df_analysis.loc[df_analysis['Dengesizlik'] < 0, 'dengesizlik_yonu'] = 'Negatif_Dengesizlik'  # Az ürettik
    
    # Her yön için detaylı analiz
    yön_analizi = df_analysis.groupby('dengesizlik_yonu').agg({
        'Dengesizlik': ['count', 'mean', 'std', 'min', 'max'],
        'Dengesizlik_Maliyeti': ['sum', 'mean', 'max'],
        'Birim_DM': ['mean', 'max'],
        'PTF': 'mean',
        'SMF': 'mean',
        'Poz_DF': 'mean',
        'Neg_DF': 'mean'
    }).round(2)
    
    print("\nYÖN BAZLI İSTATİSTİKLER:")
    print("-" * 50)
    
    for yön in ['Pozitif_Dengesizlik', 'Negatif_Dengesizlik', 'Nötr']:
        if yön in yön_analizi.index:
            data = yön_analizi.loc[yön]
            saat_sayisi = data[('Dengesizlik', 'count')]
            toplam_maliyet = data[('Dengesizlik_Maliyeti', 'sum')]
            ort_maliyet = data[('Dengesizlik_Maliyeti', 'mean')]
            
            if yön == 'Pozitif_Dengesizlik':
                print(f"FAZLA ÜRETİM (Tahmin > Gerçek):")
            elif yön == 'Negatif_Dengesizlik':
                print(f"EKSIK ÜRETİM (Tahmin < Gerçek):")
            else:
                print(f"MÜKEMMEL TAHMİN:")
            
            print(f"   Olay Sayısı: {saat_sayisi:.0f} saat")
            print(f"   Toplam Maliyet: {toplam_maliyet:,.0f} TL")
            print(f"   Ortalama Maliyet: {ort_maliyet:.0f} TL/saat")
            print(f"   Ortalama PTF: {data[('PTF', 'mean')]:.0f} TL/MWh")
            print(f"   Ortalama SMF: {data[('SMF', 'mean')]:.0f} TL/MWh")
            print()
    
    # Hangi yön daha maliyetli?
    pos_toplam = df_analysis[df_analysis['dengesizlik_yonu']=='Pozitif_Dengesizlik']['Dengesizlik_Maliyeti'].sum()
    neg_toplam = df_analysis[df_analysis['dengesizlik_yonu']=='Negatif_Dengesizlik']['Dengesizlik_Maliyeti'].sum()
    
    print("KRITIK BULGULAR:")
    print("-" * 30)
    print(f"Pozitif Dengesizlik Maliyeti: {pos_toplam:,.0f} TL")
    print(f"Negatif Dengesizlik Maliyeti: {neg_toplam:,.0f} TL")
    
    if neg_toplam > pos_toplam:
        risk_orani = neg_toplam / pos_toplam
        print(f"EKSIK ÜRETİM {risk_orani:.1f}x DAHA RİSKLİ!")
        print("Çünkü: SMF genelde PTF'den yüksek + %103 ceza çarpanı")
    else:
        risk_orani = pos_toplam / neg_toplam
        print(f"FAZLA ÜRETİM {risk_orani:.1f}x DAHA RİSKLİ!")
        print("Çünkü: PTF düşük dönemlerde fazla üretim zarar")
    
    # Fiyat spread analizi
    print(f"\nFİYAT SPREAD ANALİZİ:")
    print("-" * 30)
    ptf_ort = df['PTF'].mean()
    smf_ort = df['SMF'].mean()
    spread = smf_ort - ptf_ort
    
    print(f"Ortalama PTF: {ptf_ort:.0f} TL/MWh")
    print(f"Ortalama SMF: {smf_ort:.0f} TL/MWh")
    
    if spread > 0:
        print("SMF > PTF olduğu için NEGATİF dengesizlik daha riskli")
        print("   (Eksik üretimde yüksek SMF ile ceza yiyoruz)")
    else:
        print("PTF > SMF olduğu için POZİTİF dengesizlik daha riskli")
    
    # Saatlik risk analizi
    pozitif_saatlik = df_analysis[df_analysis['dengesizlik_yonu']=='Pozitif_Dengesizlik'].groupby(
        df_analysis['Saat'].str.replace(':00', '').astype(int)
    )['Dengesizlik_Maliyeti'].sum().sort_values(ascending=False)

    negatif_saatlik = df_analysis[df_analysis['dengesizlik_yonu']=='Negatif_Dengesizlik'].groupby(
        df_analysis['Saat'].str.replace(':00', '').astype(int)
    )['Dengesizlik_Maliyeti'].sum().sort_values(ascending=False)

    print("\nPOZİTİF DENGESİZLİKTE SAATLİK MALİYETLER (TL):")
    print(pozitif_saatlik)
    print("\nNEGATİF DENGESİZLİKTE SAATLİK MALİYETLER (TL):")
    print(negatif_saatlik)
    
    negatif_saatlik = df_analysis[df_analysis['dengesizlik_yonu']=='Negatif_Dengesizlik'].groupby(
        df_analysis['Saat'].str.replace(':00', '').astype(int)
    )['Dengesizlik_Maliyeti'].sum().sort_values(ascending=False)
    
    
    return {
        'pozitif_maliyet': pos_toplam,
        'negatif_maliyet': neg_toplam,
        'risk_yonu': 'negatif' if neg_toplam > pos_toplam else 'pozitif',
        'risk_orani': max(neg_toplam, pos_toplam) / min(neg_toplam, pos_toplam)
    }

# Analizi çalıştır
dengesizlik_analizi = melkom_dengesizlik_yonu_analizi(df)

MASLAKTEPE RES - POZİTİF vs NEGATİF DENGESİZLİK ANALİZİ

YÖN BAZLI İSTATİSTİKLER:
--------------------------------------------------
FAZLA ÜRETİM (Tahmin > Gerçek):
   Olay Sayısı: 3928 saat
   Toplam Maliyet: 9,947,939 TL
   Ortalama Maliyet: 2533 TL/saat
   Ortalama PTF: 2205 TL/MWh
   Ortalama SMF: 2089 TL/MWh

EKSIK ÜRETİM (Tahmin < Gerçek):
   Olay Sayısı: 4795 saat
   Toplam Maliyet: 7,106,481 TL
   Ortalama Maliyet: 1482 TL/saat
   Ortalama PTF: 2260 TL/MWh
   Ortalama SMF: 2253 TL/MWh

MÜKEMMEL TAHMİN:
   Olay Sayısı: 61 saat
   Toplam Maliyet: 0 TL
   Ortalama Maliyet: 0 TL/saat
   Ortalama PTF: 2274 TL/MWh
   Ortalama SMF: 2180 TL/MWh

KRITIK BULGULAR:
------------------------------
Pozitif Dengesizlik Maliyeti: 9,947,939 TL
Negatif Dengesizlik Maliyeti: 7,106,481 TL
FAZLA ÜRETİM 1.4x DAHA RİSKLİ!
Çünkü: PTF düşük dönemlerde fazla üretim zarar

FİYAT SPREAD ANALİZİ:
------------------------------
Ortalama PTF: 2236 TL/MWh
Ortalama SMF: 2179 TL/MWh
PTF > SMF olduğu için POZİTİ

In [45]:
# Pazar dinamikleri analizi
print("=== MASLAKTEPE RES - Pazar Dinamikleri Analizi ===")

# Yüksek/düşük fiyat dönemleri
high_price_threshold = df['PTF'].quantile(0.8)
low_price_threshold = df['PTF'].quantile(0.2)

high_price_perf = df[df['PTF'] >= high_price_threshold].agg({
    'URETIM': 'mean',
    'Dengesizlik': 'mean',
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(3)

low_price_perf = df[df['PTF'] <= low_price_threshold].agg({
    'URETIM': 'mean', 
    'Dengesizlik': 'mean',
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(3)

print(f"Yüksek Fiyat Dönemlerinde (PTF > {high_price_threshold:.0f} TL/MWh):")
print(f"Ortalama Üretim: {high_price_perf['URETIM']:.2f} MWh")
print(f"Ortalama Dengesizlik: {high_price_perf['Dengesizlik']:.3f} MWh")
print(f"Ortalama Birim DM: {high_price_perf['Birim_DM']:.2f} TL/MWh")

print(f"\nDüşük Fiyat Dönemlerinde (PTF < {low_price_threshold:.0f} TL/MWh):")
print(f"Ortalama Üretim: {low_price_perf['URETIM']:.2f} MWh")
print(f"Ortalama Dengesizlik: {low_price_perf['Dengesizlik']:.3f} MWh")
print(f"Ortalama Birim DM: {low_price_perf['Birim_DM']:.2f} TL/MWh")

=== MASLAKTEPE RES - Pazar Dinamikleri Analizi ===
Yüksek Fiyat Dönemlerinde (PTF > 2742 TL/MWh):
Ortalama Üretim: 25.08 MWh
Ortalama Dengesizlik: 0.556 MWh
Ortalama Birim DM: 64.46 TL/MWh

Düşük Fiyat Dönemlerinde (PTF < 1800 TL/MWh):
Ortalama Üretim: 37.73 MWh
Ortalama Dengesizlik: 0.411 MWh
Ortalama Birim DM: 108.48 TL/MWh


In [48]:
# Kayıp oranı hesabı (mükemmel senaryoya göre)
print("=== MASLAKTEPE RES - Kayıp Oranı Hesabı (Mükemmel Senaryo) ===")

# Mükemmel senaryo: Hiç dengesizlik olmasaydı kazanılacak gelir
perfect_revenue = df['URETIM'] * df['PTF']
actual_revenue = df['Net_Gelir']
revenue_loss = perfect_revenue - actual_revenue

# Kayıp oranları
total_perfect_revenue = perfect_revenue.sum()
total_actual_revenue = actual_revenue.sum()
total_revenue_loss = revenue_loss.sum()

print(f"Mükemmel Senaryo Geliri: {total_perfect_revenue:,.0f} TL")
print(f"Gerçek Net Gelir: {total_actual_revenue:,.0f} TL")
print(f"Toplam Gelir Kaybı: {total_revenue_loss:,.0f} TL")
print(f"Kayıp Oranı: {(total_revenue_loss/total_perfect_revenue)*100:.2f}%")

# Aylık kayıp oranları
monthly_loss = df.groupby('Ay').agg({
    'URETIM': 'sum',
    'PTF': lambda x: (df.loc[x.index, 'URETIM'] * df.loc[x.index, 'PTF']).sum(),
    'Net_Gelir': 'sum'
}).round(2)

monthly_loss['Mükemmel_Gelir'] = df.groupby('Ay').apply(lambda x: (x['URETIM'] * x['PTF']).sum())
monthly_loss['Gelir_Kaybi'] = monthly_loss['Mükemmel_Gelir'] - monthly_loss['Net_Gelir']
monthly_loss['Kayip_Orani'] = (monthly_loss['Gelir_Kaybi'] / monthly_loss['Mükemmel_Gelir'] * 100).round(2)

print(f"\nAylık Kayıp Oranları:")
print(monthly_loss[['Mükemmel_Gelir', 'Net_Gelir', 'Gelir_Kaybi', 'Kayip_Orani']].sort_values(by='Gelir_Kaybi', ascending=False))

=== MASLAKTEPE RES - Kayıp Oranı Hesabı (Mükemmel Senaryo) ===
Mükemmel Senaryo Geliri: 482,331,424 TL
Gerçek Net Gelir: 465,277,004 TL
Toplam Gelir Kaybı: 17,054,420 TL
Kayıp Oranı: 3.54%

Aylık Kayıp Oranları:
    Mükemmel_Gelir    Net_Gelir   Gelir_Kaybi  Kayip_Orani
Ay                                                        
12    4.751441e+07  45413032.34  2.101377e+06         4.42
10    4.541488e+07  43466963.43  1.947917e+06         4.29
4     2.959164e+07  27779877.72  1.811767e+06         6.12
2     2.934615e+07  27811380.47  1.534772e+06         5.23
9     2.480344e+07  23406311.32  1.397129e+06         5.63
1     4.837047e+07  46987807.97  1.382659e+06         2.86
11    4.525896e+07  44006663.17  1.252300e+06         2.77
3     2.007125e+07  18831998.66  1.239251e+06         6.17
5     2.583733e+07  24658545.67  1.178785e+06         4.56
6     4.400609e+07  42839957.83  1.166130e+06         2.65
8     5.587206e+07  54792585.66  1.079473e+06         1.93
7     6.624474e+07  6

/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_1865/1998647066.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_loss['Mükemmel_Gelir'] = df.groupby('Ay').apply(lambda x: (x['URETIM'] * x['PTF']).sum())


Maslaktepe RES: Mart (%6.17) ve Nisan (%6.12) en yüksek kayıp oranlarına sahip aylardır. Kışa giriş ve bahar aylarında tahmin zorlukları yaşanmış olabilir.

In [49]:
# Over/Under asimetrisi detaylı analizi
print("=== MASLAKTEPE RES - Over/Under Asimetrisi Detaylı Analizi ===")

# Pozitif (Over) ve Negatif (Under) dengesizlikleri ayır
over_production = df[df['Dengesizlik'] > 0].copy()
under_production = df[df['Dengesizlik'] < 0].copy()

print(f"Over Production Saatleri: {len(over_production)} ({len(over_production)/len(df)*100:.1f}%)")
print(f"Under Production Saatleri: {len(under_production)} ({len(under_production)/len(df)*100:.1f}%)")

# Over/Under büyüklük analizi
over_stats = {
    'Count': len(over_production),
    'Avg_Size': over_production['Dengesizlik'].mean(),
    'Median_Size': over_production['Dengesizlik'].median(),
    'Max_Size': over_production['Dengesizlik'].max(),
    'Total_Volume': over_production['Dengesizlik'].sum(),
    'Avg_Cost': over_production['Dengesizlik_Maliyeti'].mean(),
    'Total_Cost': over_production['Dengesizlik_Maliyeti'].sum()
}

under_stats = {
    'Count': len(under_production),
    'Avg_Size': under_production['Dengesizlik'].mean(),
    'Median_Size': under_production['Dengesizlik'].median(),
    'Min_Size': under_production['Dengesizlik'].min(),
    'Total_Volume': under_production['Dengesizlik'].sum(),
    'Avg_Cost': under_production['Dengesizlik_Maliyeti'].mean(),
    'Total_Cost': under_production['Dengesizlik_Maliyeti'].sum()
}

asymmetry_df = pd.DataFrame({'Over': over_stats, 'Under': under_stats}).round(3)
print(f"\nOver vs Under Asimetrisi:")
print(asymmetry_df)

# Asimetri oranları
print(f"\nAsimetri Oranları:")
print(f"Hacim Asimetrisi: {abs(over_stats['Total_Volume']) / abs(under_stats['Total_Volume']):.2f}")
print(f"Maliyet Asimetrisi: {over_stats['Total_Cost'] / abs(under_stats['Total_Cost']):.2f}")
print(f"Frekans Asimetrisi: {over_stats['Count'] / under_stats['Count']:.2f}")

=== MASLAKTEPE RES - Over/Under Asimetrisi Detaylı Analizi ===
Over Production Saatleri: 3928 (44.7%)
Under Production Saatleri: 4795 (54.6%)

Over vs Under Asimetrisi:
                     Over        Under
Count            3928.000     4795.000
Avg_Size            6.925       -5.151
Median_Size         5.030       -3.200
Max_Size           58.260          NaN
Total_Volume    27199.580   -24699.130
Avg_Cost         2532.571     1482.061
Total_Cost    9947938.768  7106481.258
Min_Size              NaN      -48.900

Asimetri Oranları:
Hacim Asimetrisi: 1.10
Maliyet Asimetrisi: 1.40
Frekans Asimetrisi: 0.82


In [50]:
# Bias & MAPE gibi tahmin doğruluğu metrikleri
print("=== MASLAKTEPE RES - Bias & MAPE Tahmin Doğruluğu Metrikleri ===")

# Sadece üretim yapılan saatleri al
active_hours = df[df['KGUP'] > 0].copy()

# BIAS (Ortalama Hata)
bias = active_hours['Dengesizlik'].mean()
bias_percentage = (bias / active_hours['KGUP'].mean()) * 100

# MAPE (Mean Absolute Percentage Error)
mape = (active_hours['Dengesizlik'].abs() / active_hours['KGUP']).mean() * 100

# MAE (Mean Absolute Error)
mae = active_hours['Dengesizlik'].abs().mean()

# RMSE (Root Mean Square Error)
rmse = (active_hours['Dengesizlik'] ** 2).mean() ** 0.5

# Tahmin doğruluğu (accuracy)
accuracy = 1 - (active_hours['Dengesizlik'].abs().sum() / active_hours['KGUP'].sum())

print(f"BIAS (Ortalama Hata): {bias:.3f} MWh ({bias_percentage:.2f}%)")
print(f"MAPE (Ortalama Mutlak Yüzde Hata): {mape:.2f}%")
print(f"MAE (Ortalama Mutlak Hata): {mae:.3f} MWh")
print(f"RMSE (Kök Ortalama Kare Hata): {rmse:.3f} MWh")
print(f"Tahmin Doğruluğu: {accuracy*100:.1f}%")

# Aylık tahmin performansı
monthly_forecast = active_hours.groupby('Ay').agg({
    'Dengesizlik': ['mean', lambda x: x.abs().mean(), lambda x: (x.abs() / active_hours.loc[x.index, 'KGUP']).mean() * 100],
    'KGUP': 'mean'
}).round(3)

monthly_forecast.columns = ['BIAS', 'MAE', 'MAPE', 'KGUP_Mean']
print(f"\nAylık Tahmin Performansı:")
print(monthly_forecast)

=== MASLAKTEPE RES - Bias & MAPE Tahmin Doğruluğu Metrikleri ===
BIAS (Ortalama Hata): 0.286 MWh (1.11%)
MAPE (Ortalama Mutlak Yüzde Hata): 52.75%
MAE (Ortalama Mutlak Hata): 5.949 MWh
RMSE (Kök Ortalama Kare Hata): 8.561 MWh
Tahmin Doğruluğu: 77.0%

Aylık Tahmin Performansı:
     BIAS    MAE     MAPE  KGUP_Mean
Ay                                  
1  -0.220  6.442   48.150     36.166
2   0.662  5.806   55.795     23.986
3   0.872  5.074  106.424     12.385
4  -0.465  7.384   53.279     28.208
5   1.515  5.180   67.794     18.238
6  -0.807  5.515   37.996     31.587
7  -2.201  6.619   30.312     37.266
8   0.027  4.935   34.650     29.611
9   0.885  5.095   56.582     13.912
10  0.472  6.142   41.259     26.711
11  1.459  5.017   46.067     26.185
12  1.293  8.136   55.168     26.262


In [53]:
print("=== MASLAKTEPE RES - RÜZGARLI Santral Fiyat Kullanım Endeksi ===")

# Fiyat seviyelerine göre üretim verimliliği (geçici değişkenler kullan)
price_quintiles = df['PTF'].quantile([0.2, 0.4, 0.6, 0.8])
price_levels = pd.cut(df['PTF'], 
                     bins=[0, price_quintiles[0.2], price_quintiles[0.4], price_quintiles[0.6], price_quintiles[0.8], float('inf')],
                     labels=['Çok_Düşük', 'Düşük', 'Orta', 'Yüksek', 'Çok_Yüksek'])

# Her fiyat seviyesinde üretim davranışı
price_efficiency = df.groupby(price_levels).agg({
    'PTF': 'mean',
    'URETIM': ['mean', 'sum', 'count'],
    'KGUP': ['mean', 'sum'],
    'Dengesizlik_Maliyeti': ['mean', 'sum']
}).round(2)

price_efficiency.columns = ['Avg_PTF', 'Avg_Production', 'Total_Production', 'Hour_Count', 'Avg_KGUP', 'Total_KGUP', 'Avg_DM', 'Total_DM']

# Fiyat Kullanım Endeksi hesapla
total_production = df['URETIM'].sum()
weighted_price = (price_efficiency['Total_Production'] * price_efficiency['Avg_PTF']).sum() / total_production
average_price = df['PTF'].mean()
price_utilization_index = weighted_price / average_price

print(f"Fiyat Seviyelerine Göre Üretim Dağılımı:")
print(price_efficiency[['Avg_PTF', 'Total_Production', 'Hour_Count', 'Avg_DM']])

print(f"\nFiyat Kullanım Endeksi: {price_utilization_index:.3f}")
print(f"(1.0'dan yüksek = yüksek fiyatlarda daha çok üretim)")

# Su rejimi stratejisi analizi
daily_production = df.groupby('Tarih').agg({
    'URETIM': 'sum',
    'PTF': 'mean',
    'Dengesizlik_Maliyeti': 'sum'
}).reset_index()

# Günlük üretim vs fiyat korelasyonu
daily_corr = daily_production['URETIM'].corr(daily_production['PTF'])
print(f"\nGünlük Üretim-Fiyat Korelasyonu: {daily_corr:.3f}")

# En değerli günlerde üretim performansı
top_price_days = daily_production.nlargest(30, 'PTF')
avg_production_high_price = top_price_days['URETIM'].mean()
avg_production_overall = daily_production['URETIM'].mean()
high_price_utilization = avg_production_high_price / avg_production_overall

print(f"En Yüksek Fiyatlı 30 Günde Ort. Üretim: {avg_production_high_price:.1f} MWh")
print(f"Genel Ortalama Günlük Üretim: {avg_production_overall:.1f} MWh")
print(f"Yüksek Fiyat Kullanım Oranı: {high_price_utilization:.2f}")

# Hidro strateji skoru
hydro_strategy_score = (price_utilization_index + daily_corr + high_price_utilization) / 3 * 100
print(f"\nRüzgar Strateji Skoru: {hydro_strategy_score:.1f}/100")
print(f"(Fiyat takibi, günlük korelasyon ve yüksek fiyat kullanımının ortalaması)")

=== MASLAKTEPE RES - RÜZGARLI Santral Fiyat Kullanım Endeksi ===
Fiyat Seviyelerine Göre Üretim Dağılımı:
            Avg_PTF  Total_Production  Hour_Count   Avg_DM
PTF                                                       
Çok_Düşük   1161.62          66204.24        1754  2522.76
Düşük       2076.86          41016.43        1756  2243.16
Orta        2374.00          37361.70        1758  1984.58
Yüksek      2629.05          39794.33        1755  1646.32
Çok_Yüksek  2939.62          44065.44        1757  1316.06

Fiyat Kullanım Endeksi: 0.949
(1.0'dan yüksek = yüksek fiyatlarda daha çok üretim)

Günlük Üretim-Fiyat Korelasyonu: -0.319
En Yüksek Fiyatlı 30 Günde Ort. Üretim: 496.0 MWh
Genel Ortalama Günlük Üretim: 624.5 MWh
Yüksek Fiyat Kullanım Oranı: 0.79

Rüzgar Strateji Skoru: 47.5/100
(Fiyat takibi, günlük korelasyon ve yüksek fiyat kullanımının ortalaması)


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_1865/1765383198.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_efficiency = df.groupby(price_levels).agg({
